# TSFRESH - Time Series FeatuRe Extraction based on Scalable Hypothesis test on Univariate Time Series

Typical workflow for TSFRESH:

    1) Load Raw Time Series
    
    2) Feature Extraction (global max, stddev, dickey fuller, fft, etc.)
    
    3) P Value Calculation
    
    4) Feature Selection (quantile, autocorrelation, cwt, etc.)

In [28]:
import warnings
warnings.filterwarnings('ignore')
def ignore_warnings(*args, **kwargs):
    pass
warnings.warn = ignore_warnings 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from tqdm import tqdm

In [29]:
#Load Chocolate dataset
url = 'https://raw.githubusercontent.com/tristanga/Machine-Learning/master/Data/Monthly%20Production%20of%20Chocolate%20-%20Australia.csv'
df = pd.read_csv(url)
df.Month = pd.to_datetime(df.Month,format='%Y-%m')
df.index = df.Month
df_chocolate = df.drop(['Month'], axis = 1)
print(df_chocolate.info())

#Load AirPassengers dataset
url = 'https://raw.githubusercontent.com/tristanga/Machine-Learning/master/Data/International%20Airline%20Passengers.csv'
df = pd.read_csv(url, sep =";")
df.Month = pd.to_datetime(df.Month,format='%Y-%m')
df.index = df.Month
df_air = df.drop(['Month'], axis = 1)
print(df_air.info())

#Load Wine dataset
url = 'https://raw.githubusercontent.com/tristanga/Data-Cleaning/master/Converting%20Time%20Series/Wine_Sales_R_Dataset.csv'
df = pd.read_csv(url)
df.Date = pd.to_datetime(df.Date,format='%Y-%m-%d')
df.index = df.Date
df_wine = df.drop(['Date'], axis = 1)
print(df_wine.info())

#Load JetRail dataset
url = 'https://raw.githubusercontent.com/tristanga/Machine-Learning/master/Data/JetRail%20Avg%20Hourly%20Traffic%20Data%20-%202012-2013.csv'
df = pd.read_csv(url)
df.Datetime = pd.to_datetime(df.Datetime,format='%Y-%m-%d %H:%M') 
df.index = df.Datetime
df_jet = df.drop(['Datetime','ID'], axis = 1)
print(df_jet.info())

#Load Portland Riders dataset
url = 'https://raw.githubusercontent.com/tristanga/Machine-Learning/master/Data/Portland%20Riders.csv'
df = pd.read_csv(url)
df.date = pd.to_datetime(df.date,format='%Y-%m-%d') 
df.index = df.date
df_portland = df.drop(['date'], axis = 1)
print(df_portland.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 458 entries, 1957-07-01 to 1995-08-01
Data columns (total 1 columns):
Volume    458 non-null int64
dtypes: int64(1)
memory usage: 7.2 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 144 entries, 1949-01-01 to 1960-12-01
Data columns (total 1 columns):
Passengers    144 non-null int64
dtypes: int64(1)
memory usage: 2.2 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 176 entries, 1980-01-01 to 1994-08-01
Data columns (total 1 columns):
Sales    176 non-null int64
dtypes: int64(1)
memory usage: 2.8 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 494 entries, 2012-08-25 to 2013-12-31
Data columns (total 1 columns):
Count    494 non-null float64
dtypes: float64(1)
memory usage: 7.7 KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 114 entries, 1973-01-01 to 1982-06-01
Data columns (total 1 columns):
riders    114 non-null int64
dtypes: int64(1)
memory usage: 1.8 KB
None


In [47]:
# Use Forecasting frame from tsfresh for rolling forecast training
df_shift, y_choco = make_forecasting_frame(df_chocolate["Volume"], kind="Volume", max_timeshift=12, rolling_direction=1)
# Extracting features from df_shift
X_choco = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)
X_choco = X_choco.loc[:, X_choco.apply(pd.Series.nunique) != 1] 
X_choco.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.62s/it]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"var"__ch

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


variable,value__abs_energy,value__absolute_sum_of_changes,"value__agg_autocorrelation__f_agg_""mean""","value__agg_autocorrelation__f_agg_""median""","value__agg_autocorrelation__f_agg_""var""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""",...,value__symmetry_looking__r_0.75,value__symmetry_looking__r_0.8,value__symmetry_looking__r_0.8500000000000001,value__symmetry_looking__r_0.9,value__symmetry_looking__r_0.9500000000000001,value__time_reversal_asymmetry_statistic__lag_1,value__time_reversal_asymmetry_statistic__lag_2,value__time_reversal_asymmetry_statistic__lag_3,value__variance,value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
1957-08-01,8720209.0,0.0,0.000000,0.000000,0.000000,6174.0,-1.0,-777.0,5624.166667,-0.3616,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0000,0.0
1957-09-01,15663434.0,318.0,-1.000000,-1.000000,0.000000,6174.0,-1.0,-777.0,5624.166667,-0.3616,...,1.0,1.0,1.0,1.0,1.0,0.000000e+00,0.000000e+00,0.0,25281.0000,1.0
1957-10-01,21442650.0,549.0,-0.745849,-0.745849,0.543977,6174.0,-1.0,-777.0,5624.166667,-0.3616,...,1.0,1.0,1.0,1.0,1.0,-7.749517e+09,0.000000e+00,0.0,50654.0000,1.0
1957-11-01,27265219.0,558.0,-0.602167,-0.760354,0.446139,6174.0,-1.0,-777.0,5624.166667,-0.3616,...,1.0,1.0,1.0,1.0,1.0,-5.221787e+09,0.000000e+00,0.0,49803.1875,1.0
1957-12-01,31827715.0,835.0,-0.653685,-0.344511,0.922409,6174.0,-1.0,-777.0,5624.166667,-0.3616,...,1.0,1.0,1.0,1.0,1.0,-4.459840e+09,-9.995142e+09,0.0,74475.7600,1.0


In [48]:
# Use Forecasting frame from tsfresh for rolling forecast training
df_shift, y_air = make_forecasting_frame(df_air["Passengers"], kind="Passengers", max_timeshift=12, rolling_direction=1)
# Extracting features from df_shift
X_air = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)
X_air = X_air.loc[:, X_air.apply(pd.Series.nunique) != 1] 
X_air.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.09it/s]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"var"__ch

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


variable,value__abs_energy,value__absolute_sum_of_changes,"value__agg_autocorrelation__f_agg_""mean""","value__agg_autocorrelation__f_agg_""median""","value__agg_autocorrelation__f_agg_""var""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""",...,value__symmetry_looking__r_0.75,value__symmetry_looking__r_0.8,value__symmetry_looking__r_0.8500000000000001,value__symmetry_looking__r_0.9,value__symmetry_looking__r_0.9500000000000001,value__time_reversal_asymmetry_statistic__lag_1,value__time_reversal_asymmetry_statistic__lag_2,value__time_reversal_asymmetry_statistic__lag_3,value__variance,value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
1949-02-01,12544.0,0.0,0.000000,0.000000,0.000000,302.0,-1.0,-33.0,275.083333,-0.074027,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1949-03-01,26468.0,6.0,-1.000000,-1.000000,0.000000,302.0,-1.0,-33.0,275.083333,-0.074027,...,1.0,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,9.000000,1.0
1949-04-01,43892.0,20.0,-0.724684,-0.724684,0.454344,302.0,-1.0,-33.0,275.083333,-0.074027,...,1.0,1.0,1.0,1.0,1.0,575840.000000,0.0,0.0,70.222222,1.0
1949-05-01,60533.0,23.0,-0.558727,-0.982873,0.394764,302.0,-1.0,-33.0,275.083333,-0.074027,...,1.0,1.0,1.0,1.0,1.0,467242.000000,0.0,0.0,65.687500,1.0
1949-06-01,75174.0,31.0,-0.234304,-0.158654,0.269117,302.0,-1.0,-33.0,275.083333,-0.074027,...,1.0,1.0,1.0,1.0,1.0,191825.666667,276804.0,0.0,53.040000,1.0


In [49]:
# Use Forecasting frame from tsfresh for rolling forecast training
df_shift, y_wine = make_forecasting_frame(df_wine["Sales"], kind="Sales", max_timeshift=12, rolling_direction=1)
# Extracting features from df_shift
X_wine = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)
X_wine = X_wine.loc[:, X_wine.apply(pd.Series.nunique) != 1] 
X_wine.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.16s/it]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"var"__ch

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


variable,value__abs_energy,value__absolute_sum_of_changes,"value__agg_autocorrelation__f_agg_""mean""","value__agg_autocorrelation__f_agg_""median""","value__agg_autocorrelation__f_agg_""var""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""",...,value__symmetry_looking__r_0.75,value__symmetry_looking__r_0.8,value__symmetry_looking__r_0.8500000000000001,value__symmetry_looking__r_0.9,value__symmetry_looking__r_0.9500000000000001,value__time_reversal_asymmetry_statistic__lag_1,value__time_reversal_asymmetry_statistic__lag_2,value__time_reversal_asymmetry_statistic__lag_3,value__variance,value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
1980-02-01,2.290985e+08,0.0,0.000000,0.000000,0.000000,35110.0,-1.0,-7615.0,33174.25,-0.545833,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0
1980-03-01,5.090918e+08,1597.0,-1.000000,-1.000000,0.000000,35110.0,-1.0,-7615.0,33174.25,-0.545833,...,1.0,1.0,1.0,1.0,1.0,0.000000e+00,0.000000e+00,0.0,6.376022e+05,1.0
1980-04-01,9.097320e+08,4880.0,-0.730867,-0.730867,0.479697,35110.0,-1.0,-7615.0,33174.25,-0.545833,...,1.0,1.0,1.0,1.0,1.0,2.870408e+12,0.000000e+00,0.0,4.126989e+06,1.0
1980-05-01,1.223305e+09,7188.0,-0.380883,-0.224075,0.192913,35110.0,-1.0,-7615.0,33174.25,-0.545833,...,1.0,1.0,1.0,1.0,1.0,1.771273e+12,0.000000e+00,0.0,3.127223e+06,1.0
1980-06-01,1.547990e+09,7499.0,-0.302778,-0.312579,0.068714,35110.0,-1.0,-7615.0,33174.25,-0.545833,...,1.0,1.0,1.0,1.0,1.0,7.325060e+11,1.913247e+12,0.0,2.563431e+06,1.0


In [50]:
# Use Forecasting frame from tsfresh for rolling forecast training
df_shift, y_jet = make_forecasting_frame(df_jet["Count"], kind="Count", max_timeshift=12, rolling_direction=1)
# Extracting features from df_shift
X_jet = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)
X_jet = X_jet.loc[:, X_jet.apply(pd.Series.nunique) != 1] 
X_jet.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.79s/it]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"var"__ch

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


variable,value__abs_energy,value__absolute_sum_of_changes,"value__agg_autocorrelation__f_agg_""mean""","value__agg_autocorrelation__f_agg_""median""","value__agg_autocorrelation__f_agg_""var""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""",...,value__symmetry_looking__r_0.75,value__symmetry_looking__r_0.8,value__symmetry_looking__r_0.8500000000000001,value__symmetry_looking__r_0.9,value__symmetry_looking__r_0.9500000000000001,value__time_reversal_asymmetry_statistic__lag_1,value__time_reversal_asymmetry_statistic__lag_2,value__time_reversal_asymmetry_statistic__lag_3,value__variance,value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
2012-08-26,10.027778,0.000000,0.000000,0.000000,0.000000,55.166667,-1.0,-11.416667,45.715278,-0.511205,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2012-08-27,23.472222,0.500000,-1.000000,-1.000000,0.000000,55.166667,-1.0,-11.416667,45.715278,-0.511205,...,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.0,0.062500,0.0
2012-08-28,30.145833,1.583333,-0.394685,-0.394685,0.099821,55.166667,-1.0,-11.416667,45.715278,-0.511205,...,1.0,1.0,1.0,1.0,1.0,-12.298611,0.000000,0.0,0.195988,0.0
2012-08-29,35.986111,1.750000,-0.429472,-0.460993,0.187446,55.166667,-1.0,-11.416667,45.715278,-0.511205,...,1.0,1.0,1.0,1.0,1.0,-15.971354,0.000000,0.0,0.244792,0.0
2012-08-30,42.236111,1.833333,-0.393790,-0.487490,0.209361,55.166667,-1.0,-11.416667,45.715278,-0.511205,...,1.0,1.0,1.0,1.0,1.0,-10.988812,-9.759259,0.0,0.229444,0.0


In [51]:
# Use Forecasting frame from tsfresh for rolling forecast training
df_shift, y_portland = make_forecasting_frame(df_portland["riders"], kind="riders", max_timeshift=12, rolling_direction=1)
# Extracting features from df_shift
X_portland = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)
X_portland = X_portland.loc[:, X_portland.apply(pd.Series.nunique) != 1] 
X_portland.head()

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.31it/s]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"min"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"var"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"var"__ch

 'value__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.


variable,value__abs_energy,value__absolute_sum_of_changes,"value__agg_autocorrelation__f_agg_""mean""","value__agg_autocorrelation__f_agg_""median""","value__agg_autocorrelation__f_agg_""var""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","value__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""intercept""","value__agg_linear_trend__f_agg_""max""__chunk_len_5__attr_""rvalue""",...,value__symmetry_looking__r_0.75,value__symmetry_looking__r_0.8,value__symmetry_looking__r_0.8500000000000001,value__symmetry_looking__r_0.9,value__symmetry_looking__r_0.9500000000000001,value__time_reversal_asymmetry_statistic__lag_1,value__time_reversal_asymmetry_statistic__lag_2,value__time_reversal_asymmetry_statistic__lag_3,value__variance,value__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
1973-02-01,4.199040e+09,0.0,0.000000,0.000000,0.000000,123400.0,-1.0,-2300.0,119741.666667,0.45142,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0
1973-03-01,8.372200e+09,200.0,-1.000000,-1.000000,0.000000,123400.0,-1.0,-2300.0,119741.666667,0.45142,...,1.0,1.0,1.0,1.0,1.0,0.000000e+00,0.000000e+00,0.0,10000.000000,1.0
1973-04-01,1.245541e+10,900.0,-0.703358,-0.703358,0.372191,123400.0,-1.0,-2300.0,119741.666667,0.45142,...,1.0,1.0,1.0,1.0,1.0,-7.482618e+12,0.000000e+00,0.0,148888.888889,1.0
1973-05-01,1.673257e+10,2400.0,-0.181312,-0.263617,0.142050,123400.0,-1.0,-2300.0,119741.666667,0.45142,...,1.0,1.0,1.0,1.0,1.0,-4.185090e+11,0.000000e+00,0.0,286875.000000,1.0
1973-06-01,2.070157e+10,4800.0,-0.293436,-0.295401,0.269634,123400.0,-1.0,-2300.0,119741.666667,0.45142,...,1.0,1.0,1.0,1.0,1.0,-2.768784e+12,-1.469956e+13,0.0,678400.000000,1.0
